In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
import zcode.math as zmath

# Construct a population of binaries

In [ ]:
NUM = 1e6
MASS_EXTR = [1e6, 1e10]
# TMAX = (20.0 * YR)
# NFREQS = 100
TMAX = (2.0 * YR)
NFREQS = 1000

In [ ]:
fobs_gw = np.arange(1, NFREQS+1) / TMAX

masses = zmath.random_power(MASS_EXTR, -3, NUM) * MSOL
redz = 0.1
mrat = 0.3

fig, ax = plot.figax()
kale.dist1d((masses/MSOL), carpet=False)
plt.show()

# Calculate the strain directly 

# Semi-Analytic (SA) Calculation

The GWB characteristic strain spectrum can be calculated **semi-analytically** using a volumetric number-density of sources $n(M, q, z) = dN/dV_c$, as [Phinney 2001, Eq. 5] or [Enoki & Nagashima 2007, Eq. 3.6]:

$$ h_c^2 = \frac{4G}{\pi c^2 f} \int dM \, dq \, dz \, \frac{d^3 n(M, q, z)}{dM \, dq \, dz} \, \left( \frac{dE_{GW}(M, q)}{d f_r}\right)_{f_r = f(1+z)}$$

Assuming circular, GW-driven orbits, this can be simplified to [Enoki & Nagashima 2007, Eq.3.11]:

$$ h_c^2 = \frac{4\pi}{3 c^2} (\pi f)^{-4/3} \int dM \, dq \, dz \, \frac{d^3 n(M, q, z)}{dM \, dq \, dz} \, \frac{(G\mathcal{M})^{5/3}}{(1+z)^{1/3}}$$

Construct a Number-Density

In [ ]:
NBINS = 100
mbin_edges = zmath.spacing(masses, 'log', NBINS+1)
mbin_cents = 0.5 * (mbin_edges[:-1] + mbin_edges[1:])
# calculate comoving-volume
vcom = cosmo.comoving_volume(redz).cgs.value

SHAPE = (NBINS,)
ndens = np.zeros(SHAPE)
ndens, *_ = sp.stats.binned_statistic(masses, None, statistic='count', bins=mbin_edges)
ndens /= np.diff(mbin_edges)
ndens /= vcom

fig, ax = plot.figax()
plot.draw_hist_steps(ax, mbin_edges/MSOL, ndens*MSOL*(MPC**3))
plt.show()

Calculate GWB assuming circular, GW-drive evolution

In [ ]:
mchirp_edges = utils.chirp_mass_mtmr(mbin_edges, mrat)
mchirp_cents = 0.5 * (mchirp_edges[:-1] + mchirp_edges[1:])
integrand = ndens * np.power(NWTG * mchirp_cents, 5.0/3.0) * np.power(1+redz, -1.0/3.0)

gwb_sa = ((4.0 * np.pi) / (3 * SPLC**2)) * np.power(np.pi*fobs_gw, -4.0/3.0) * np.sum(integrand * np.diff(mbin_edges))
gwb_sa = np.sqrt(gwb_sa)

fig, ax = plot.figax()
xx = fobs_gw * YR
ax.plot(xx, gwb_sa)
plt.show()

# Monte Carlo (MC) Calculation

The GWB can also be calculated explicitly from the full population of binaries in the universe [Sesana et al. 2008, Eq.~10], 
$$h_c^2(f) = \int_0^\infty \!\! dz \; \frac{d^2 N}{dz \, d\ln f_r} \; h^2(f_r),$$

where the spectral GW strain (*not* characteristic strain) for a circular binary is,

$$h(f_r) = \frac{8}{10^{1/2}} \frac{(G\mathcal{M})^{5/3}}{c^4 d_c} (2\pi f_r)^{2/3}.$$

From [Sesana et al. 2008, Eq.6] we can write,

$$\frac{d^2 N}{dz \, d\ln f_r} = \frac{d n_c}{dz} \frac{dz}{dt} \frac{dt}{d\ln f_r} \frac{d V_c}{dz}.$$

The standard cosmographic relations are [Hogg 1999],

$$\frac{dz}{dt} = H_0 (1+z) E(z) \\
    \frac{d V_c}{dz} = 4\pi \frac{c}{H_0} \frac{d_c^2}{E(z)} \\
    d_L = d_c \, (1+z)$$

Combining these, we obtain:

$$h_c^2(f) = \int_0^\infty \!\! dz \; \frac{dn_c}{dz} \, h^2(f_r) \, 4\pi c \, d_c^2 (1+z) \, \frac{f_r}{df_r / dt}.$$

The hardening timescale for a circular, GW-driven binary is:

$$\tau_{GW} \equiv \frac{f_r}{\left[df_r/dt\right]_{GW}} = \frac{5}{96} \frac{c^5}{(G \mathcal{M})^{5/3}} (2\pi f_r)^{-8/3}.$$

Plugging this in to the previous relation gives:

$$h_c^2(f) = \frac{20\pi c^6}{96} \int_0^\infty \!\! dz \; \frac{dn_c}{dz} \, h^2(f_r) \, \frac{d_c^2 (1+z)}{(G \mathcal{M})^{5/3}} (2\pi f_r)^{-8/3}.$$

<!-- When taking the strain to be due to a circular binary:

$$ h_c^2(f) = \int_0^\infty \!\! dz \; \frac{256 \pi}{10 c^7} \frac{dn_c}{dz} \, \frac{f_r}{df_r / dt} \, (1+z) \, (G \mathcal{M})^{10/3} \, (2\pi f_r)^{4/3}.$$

And for GW-driven evolution,

$$h_c^2(f) = \int_0^\infty \!\! dz \; \frac{4 \pi}{3 c^2} \frac{dn_c}{dz}  \, \frac{(G \mathcal{M})^{5/3}}{(1+z)^{1/3}} \, (2\pi f_r)^{-4/3}.$$ -->

In [ ]:

dcom = cosmo.comoving_distance(redz).cgs.value
frst_orb = fobs_gw[np.newaxis, :] * (1.0 + redz) / 2.0

hs = (8.0 / np.sqrt(10)) * np.power(NWTG * mchirp_cents, 5.0/3.0) / (dcom * (SPLC**4))
hs = hs[:, np.newaxis] * np.power(2*np.pi*frst_orb, 2.0/3.0) 

integrand = ((20*np.pi*(SPLC**6))/96) * ndens * np.diff(mbin_edges)
integrand *= (dcom**2) * (1.0 + redz) * np.power(NWTG * mchirp_cents, -5.0/3.0)
integrand = integrand[:, np.newaxis] * np.power(2.0*np.pi*frst_orb, -8.0/3.0)

sepa_isco = 6 * NWTG * mbin_cents / SPLC**2
frst_orb_isco = utils.kepler_freq_from_sepa(mbin_cents, sepa_isco)
bads = frst_orb > frst_orb_isco[:, np.newaxis]
merged = np.ones_like(integrand)
merged[bads] = 0.0

integrand = np.random.poisson(integrand[..., np.newaxis], integrand.shape + (20,)) * merged[..., np.newaxis]
gwb_mc = np.sum(integrand * (hs**2)[..., np.newaxis], axis=0)

gwb_mc = np.sqrt(gwb_mc)

In [ ]:
fig, ax = plot.figax()
xx = fobs_gw * YR
# ax.plot(xx, gwb_mc[:, np.random.choice(20, 5, replace=False)], alpha=0.2)
ax.plot(xx, np.median(gwb_mc, axis=-1), lw=0.5)
ax.fill_between(xx, *np.percentile(gwb_mc, [25, 75], axis=-1), alpha=0.5)


ax.plot(xx, gwb_sa, 'k--')
plt.show()